In [1]:
import struct

In [71]:
!pipenv install cStringIO

Installing cStringIO…

Error:  An error occurred while installing cStringIO!
  ERROR: Could not find a version that satisfies the requirement cstringio (from versions: none)
ERROR: No matching distribution found for cstringio



In [2]:
from io import StringIO

def uncompress_lz77(compressed):
    """Decompress a list of output ks to a string."""
 
    # Build the dictionary.
    dict_size = 256
#     dictionary = dict((i, chr(i)) for i in xrange(dict_size))
    dictionary = {i: chr(i) for i in range(dict_size)}
    # in Python 3: dictionary = {i: chr(i) for i in range(dict_size)}
 
    # use StringIO, otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = StringIO()
    w = chr(compressed[0])
    compressed = compressed[1:]
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)
 
        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
 
        w = entry
    return result.getvalue()
 

In [3]:
def uncompress_lz77(data):
#     print(data[:100])
#     data = str(data, encoding='utf_16')
#     print(data[:100])
#     print('--\n\n')
    
    length = len(data);
    offset = 0;   # Current offset into data
    # char;      # Character being examined
    # ord;      # Ordinal of $char
    # lz77;      # 16-bit Lempel-Ziv 77 length-offset pair
    # lz77offset;   # LZ77 offset
    # lz77length;   # LZ77 length
    # lz77pos;    # Position inside $lz77length
    text = '';   # Output (uncompressed) text
    # textlength;   # Length of uncompressed text during LZ77 pass
    # textpos;    # Position inside $text during LZ77 pass

    while offset < length:
        # char = substr($data,$offset++,1);
        char = data[offset];
        offset += 1;
#         ord_ = ord(char);
        ord_ = char

        # print " ".join([repr(char), hex(ord_)])

        # The long if-elsif chain is the best logic for $ord handling
        ## no critic (Cascading if-elsif chain)
        if (ord_ == 0):
            # Nulls are literal
            text += chr(char);
        elif (ord_ <= 8):
            # Next $ord bytes are literal
            text += str(data[offset:offset+ord_]) 
            # text .=substr($data,$offset,ord);
            offset += ord_;
        elif (ord_ <= 0x7f):
            # Values from 0x09 through 0x7f are literal
            text += chr(char);
        elif (ord_ <= 0xbf):
            # Data is LZ77-compressed

            # From Wikipedia:
            # "A length-distance pair is always encoded by a two-byte
            # sequence. Of the 16 bits that make up these two bytes,
            # 11 bits go to encoding the distance, 3 go to encoding
            # the length, and the remaining two are used to make sure
            # the decoder can identify the first byte as the beginning
            # of such a two-byte sequence."

            offset += 1;
            if (offset > len(data)):
                print("WARNING: offset to LZ77 bits is outside of the data: %d" % offset);
                return text;

            lz77, = struct.unpack('>H', data[offset-2:offset])

            # Leftmost two bits are ID bits and need to be dropped
            lz77 &= 0x3fff;

            # Length is rightmost 3 bits + 3
            lz77length = (lz77 & 0x0007) + 3;

            # Remaining 11 bits are offset
            lz77offset = lz77 >> 3;
            if (lz77offset < 1):
                print("WARNING: LZ77 decompression offset is invalid!");
                return text;

            # Getting text from the offset is a little tricky, because
            # in theory you can be referring to characters you haven't
            # actually decompressed yet. You therefore have to check
            # the reference one character at a time.
            textlength = len(text);
            for lz77pos in range(lz77length): # for($lz77pos = 0; $lz77pos < $lz77length; $lz77pos++)
                textpos = textlength - lz77offset;
                if (textpos < 0):
                    print("WARNING: LZ77 decompression reference is before"+" beginning of text! %x" % lz77);
                    return;

                text += text[textpos:textpos+1]; #text .= substr($text,$textpos,1);
                textlength+=1;
        else:
            # 0xc0 - 0xff are single characters (XOR 0x80) preceded by
            # a space
            text += ' ' + chr(ord_ ^ 0x80);
    return text;


In [4]:
def toDict(tuples):
    resultsDict = {}
    for field, value in tuples:
        if len(field) > 0 and field[0] != "-":
            resultsDict[field] = value
    return resultsDict;

In [5]:
class Mobi:
    header_header_fmt = '>32shhIIIIII4s4sIIH'
    header_header_len = struct.calcsize(header_header_fmt)
    header_fields = [
        "name",
        "attributes",
        "version",
        "created",
        "modified",
        "backup",
        "modnum",
        "appInfoId",
        "sortInfoID",
        "type",
        "creator",
        "uniqueIDseed",
        "nextRecordListID",
        "number of records"
    ]
    
    record_header_fmt = '>II'
    record_header_len = struct.calcsize('>II')
    record_fields = [
        "record Data Offset",
        "UniqueID",
    ]
    
    palm_header_fmt = '>HHIHHHH'
    palm_header_len = struct.calcsize(palm_header_fmt)
    palm_fields = [
        "Compression",
        "Unused",
        "text length",
        "record count",
        "record size",
        "Encryption Type",
        "Unknown"
    ]
    
    mobi_header_fmt = '> IIII II 40s III IIIII IIII I 36s IIII 8s HHIIIII'
    mobi_header_len = struct.calcsize(mobi_header_fmt)

    mobi_fields = [
        "identifier",
        "header length",
        "Mobi type",
        "text Encoding",

        "Unique-ID",
        "Generator version",

        "-Reserved",

        "First Non-book index",
        "Full Name Offset",
        "Full Name Length",

        "Language",
        "Input Language",
        "Output Language",
        "Format version",
        "First Image index",

        "First Huff Record",
        "Huff Record Count",
        "First DATP Record",
        "DATP Record Count",

        "EXTH flags",

        "-36 unknown bytes, if Mobi is long enough",

        "DRM Offset",
        "DRM Count",
        "DRM Size",
        "DRM Flags",

        "-Usually Zeros, unknown 8 bytes",

        "-Unknown",
        "Last Image Record",
        "-Unknown",
        "FCIS record",
        "-Unknown",
        "FLIS record",
        "Unknown"
    ]
    
    exth_header_fmt = '>III'
    exth_header_len = struct.calcsize(exth_header_fmt)

    exth_fields = [
        'identifier',
        'header length',
        'record Count'
    ]
    
    def __init__(self, filename):
        try:
            if isinstance(filename, str):
                self.f = open(filename, "rb");
            else:
                self.f = filename;
        except IOError as e:
            sys.stderr.write("Could not open %s! " % filename);
            raise e;
        self.offset = 0;

    def __iter__(self):
        if not self.config: 
            return
        for record in range(1, self.config['mobi']['First Non-book index'] - 1):
            yield self.readRecord(record);

    def readRecord(self, recordnum, disable_compression=False):
        if self.config:
            if self.config['palmdoc']['Compression'] == 1 or disable_compression:
                return self.contents[self.records[recordnum]['record Data Offset']:self.records[recordnum+1]['record Data Offset']];
            elif self.config['palmdoc']['Compression'] == 2:
                result = uncompress_lz77(self.contents[self.records[recordnum]['record Data Offset']:self.records[recordnum+1]['record Data Offset']-self.config['mobi']['extra bytes']])
                return result
        
    def parse(self):
        self.contents = self.f.read();
        self.header = self.parseHeader();
        self.records = self.parseRecordInfoList();
        self.readRecord0()
        
    def parseHeader(self):
        # unpack header, zip up into list of tuples
        results = zip(
            Mobi.header_fields,
            struct.unpack(
                Mobi.header_header_fmt,
                self.contents[self.offset:self.offset+Mobi.header_header_len]))

        # increment offset into file
        self.offset += Mobi.header_header_len

        # convert tuple array to dictionary
        resultsDict = toDict(results);

        return resultsDict
    
    def parseRecordInfoList(self):
        records = {};
        # read in all records in info list
        for recordID in range(self.header['number of records']):
            
            # create tuple with info
            results = zip(
                Mobi.record_fields,
                struct.unpack(
                    Mobi.record_header_fmt,
                    self.contents[self.offset:self.offset+Mobi.record_header_len]))

            # increment offset into file
            self.offset += Mobi.record_header_len

            # convert tuple to dictionary
            resultsDict = toDict(results);

            # futz around with the unique ID record, as the uniqueID's top 8 bytes are
            # really the "record attributes":
            resultsDict['record Attributes'] = (resultsDict['UniqueID'] & 0xFF000000) >> 24;
            resultsDict['UniqueID'] = resultsDict['UniqueID'] & 0x00FFFFFF;

            # store into the records dict
            records[resultsDict['UniqueID']] = resultsDict;

        return records;
    
    def readRecord0(self):
        palmdocHeader = self.parsePalmDOCHeader();
        MobiHeader = self.parseMobiHeader();
        exthHeader = None
        if MobiHeader['Has EXTH Header']:
            exthHeader = self.parseEXTHHeader();

        self.config = {
            'palmdoc': palmdocHeader,
            'mobi' : MobiHeader,
            'exth' : exthHeader
        }
        
    def parsePalmDOCHeader(self):
        offset = self.records[0]['record Data Offset'];
        # create tuple with info
        results = zip(
            Mobi.palm_fields,
            struct.unpack(Mobi.palm_header_fmt, self.contents[offset:offset+Mobi.palm_header_len]))

        # convert tuple array to dictionary
        resultsDict = toDict(results);

        self.offset = offset+Mobi.palm_header_len;
        return resultsDict
    
    def parseMobiHeader(self):
        # unpack header, zip up into list of tuples
        results = zip(
            Mobi.mobi_fields,
            struct.unpack(Mobi.mobi_header_fmt, self.contents[self.offset:self.offset+Mobi.mobi_header_len]))

        # convert tuple array to dictionary
        resultsDict = toDict(results);

        resultsDict['Start Offset'] = self.offset;

        resultsDict['Full Name'] = (self.contents[
            self.records[0]['record Data Offset'] + resultsDict['Full Name Offset'] :
            self.records[0]['record Data Offset'] + resultsDict['Full Name Offset'] + resultsDict['Full Name Length']])

        resultsDict['Has DRM'] = resultsDict['DRM Offset'] != 0xFFFFFFFF;

        resultsDict['Has EXTH Header'] = (resultsDict['EXTH flags'] & 0x40) != 0;

        self.offset += resultsDict['header length'];

        def onebits(x, width=16):
            return len([x for x in (str((x>>i)&1) for i in range(width-1,-1,-1)) if x == '1'])

        resultsDict['extra bytes'] = 2*onebits(struct.unpack(">H", self.contents[self.offset-2:self.offset])[0] & 0xFFFE)

        return resultsDict;
    
    def parseEXTHHeader(self):
        # unpack header, zip up into list of tuples
        results = zip(
            Mobi.exth_fields,
            struct.unpack(Mobi.exth_header_fmt, self.contents[self.offset:self.offset+Mobi.exth_header_len]))

        # convert tuple array to dictionary
        resultsDict = toDict(results);

        self.offset += Mobi.exth_header_len;
        resultsDict['records'] = {};
        for record in range(resultsDict['record Count']):
            recordType, recordLen = struct.unpack(">II", self.contents[self.offset:self.offset+8]);
            recordData = self.contents[self.offset+8:self.offset+recordLen];
            resultsDict['records'][recordType] = recordData;
            self.offset += recordLen;

        return resultsDict;
    

In [12]:
book = Mobi('../data/hugo_les_miserables_cosette.prc')
book.parse()

In [15]:
book.records

{0: {'record Data Offset': 1792, 'UniqueID': 0, 'record Attributes': 0},
 1: {'record Data Offset': 4244, 'UniqueID': 1, 'record Attributes': 0},
 2: {'record Data Offset': 6506, 'UniqueID': 2, 'record Attributes': 0},
 3: {'record Data Offset': 8728, 'UniqueID': 3, 'record Attributes': 0},
 4: {'record Data Offset': 10999, 'UniqueID': 4, 'record Attributes': 0},
 5: {'record Data Offset': 13301, 'UniqueID': 5, 'record Attributes': 0},
 6: {'record Data Offset': 15558, 'UniqueID': 6, 'record Attributes': 0},
 7: {'record Data Offset': 17970, 'UniqueID': 7, 'record Attributes': 0},
 8: {'record Data Offset': 20295, 'UniqueID': 8, 'record Attributes': 0},
 9: {'record Data Offset': 22564, 'UniqueID': 9, 'record Attributes': 0},
 10: {'record Data Offset': 24834, 'UniqueID': 10, 'record Attributes': 0},
 11: {'record Data Offset': 27238, 'UniqueID': 11, 'record Attributes': 0},
 12: {'record Data Offset': 29632, 'UniqueID': 12, 'record Attributes': 0},
 13: {'record Data Offset': 32043, '

In [8]:
ord('<')

60

In [9]:
??str

Init signature: str(self, /, *args, **kwargs)
Docstring:     
str(object='') -> str
str(bytes_or_buffer[, encoding[, errors]]) -> str

Create a new string object from the given object. If encoding or
errors is specified, then the object must expose a data buffer
that will be decoded using the given encoding and error handler.
Otherwise, returns the result of object.__str__() (if defined)
or repr(object).
encoding defaults to sys.getdefaultencoding().
errors defaults to 'strict'.
Type:           type


In [10]:
test0 = "<toto>"
test1 = b'<toto>'

test1[0]

60

In [11]:
for record in book:
    print(record)

<html><head><guide><reference type="toc" title="Table of Contents" filepos=0000809353 /></guide></head><body><div><p height="4em" width="0pt" align="center"><font size="7"><b>Victor Hugo</b></font></p><p height="4em" width="0pt" align="center"><font size="7"><b>LES MISb'\xc3\x89'RAB\xct></p><p hISbr></="7"><b>LES MISb'b'\xc2\xa0'hISbr></="7"><b>LES MISb'b'\xc2\xaTome II b'\xe2\x80\x93' COSETTE><b>LES MIt" align="center"><font size="7"><b>LES MISb'\xc3\2\xaTome I5 b'\xeexte annotb'\xc3\xa9' par Guy Rosa, professeur b'\xc3\xa0' lb'\xe2\x80\x99'UniversifessParis-Dterrotont size="\x93' COSE3\xa9' par>1862size="7"><"7"><b>/"><b>mbp:pagebreakn="c"><b>"><b>VictoISb'\ I5 b\xaTome I5 b'\xeexte annotb'img recindexme "0p2 b'\xeextbaseline"ictoISb'\100% b\xaTome100% n="c<"7"><b>/"><b>mbp:pagebreakn="c"><b>"><b>VictoISb'\xc3\2\xaTome I5 b'\xeexte annotb'\xc3\x><b>LES MI>Livre0' emier="cenWannolofont size="7"><"7"><b>/"><b>mbp:pagebreakn="c"><b>"><b>an="VictoISb'\xc3\2\xaTome I5 b'\xeexte annotb'\xc

In [69]:
book.config

{'palmdoc': {'Compression': 2,
  'Unused': 0,
  'text length': 824567,
  'record count': 202,
  'record size': 4096,
  'Encryption Type': 0,
  'Unknown': 0},
 'mobi': {'identifier': 1297039945,
  'header length': 232,
  'Mobi type': 2,
  'text Encoding': 65001,
  'Unique-ID': 3647890660,
  'Generator version': 6,
  'First Non-book index': 204,
  'Full Name Offset': 856,
  'Full Name Length': 35,
  'Language': 12,
  'Input Language': 0,
  'Output Language': 0,
  'Format version': 6,
  'First Image index': 207,
  'First Huff Record': 0,
  'Huff Record Count': 0,
  'First DATP Record': 0,
  'DATP Record Count': 0,
  'EXTH flags': 80,
  'DRM Offset': 4294967295,
  'DRM Count': 0,
  'DRM Size': 0,
  'DRM Flags': 0,
  'Last Image Record': 210,
  'FCIS record': 212,
  'FLIS record': 211,
  'Unknown': 1,
  'Start Offset': 1808,
  'Full Name': b'Les Mis\xc3\xa9rables - Tome II - Cosette',
  'Has DRM': False,
  'Has EXTH Header': True,
  'extra bytes': 8},
 'exth': {'identifier': 1163416648,
  '